<a href="https://colab.research.google.com/github/tyleretheridge/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/Tyler_Etheridge_LS_DS_431_RNN_and_LSTM_Assignment_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

## Data

In [22]:
import requests
import pandas as pd
import numpy as np
import random
import sys
import os

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

In [23]:
# Set data url
url = "https://www.gutenberg.org/files/100/100-0.txt"

# Get request
r = requests.get(url)
# Set data encoding to apparent encoding from source
r.encoding = r.apparent_encoding

In [24]:
# Save text returned from requests
data = r.text

# Split text aggregate into list of strings by newline/return
data = data.split('\r\n')

# Use list comprehension to make table of contents
toc = [l.strip() for l in data[44:130:2]]

# Redefine data, skipping toc
data = data[135:]

In [25]:
# Fixing titles in ToC manually
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

In [26]:
# Create dictionary that ids each title and gives default start position
# 0: {'start': -99, 'title': 'ALL’S WELL THAT ENDS WELL'},
locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Update 'start' in title index dictionary
# 0: {'start': 2912, 'title': 'ALL’S WELL THAT ENDS WELL'}
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})

In [27]:
# Create dataframe from the locations dictionary
df_toc = pd.DataFrame.from_dict(locations, orient='index')
# Create end column using "start - 1" from next row 
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
# Add end column for final row
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')
# Add text data to df using start and end positoin to access data
df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [28]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


## Embeddings

In [29]:
# Concatenate all text
text = " ".join(data)
type(text)

str

In [30]:
# Create list of characters in 'text'
chars = list(set(text))
# Get number of unique characters in list
len(chars)

104

In [31]:
# Create dictionaries that contain lookups for chars

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [32]:
# Create numerical representations of characters/text in list
encoded = [char_int[c] for c in text]

# Encode sequence len
maxlen = 40
# Positional increment
step = 5

# List of lists w/ 40chars
sequences = []
# ???
# One element for each sequence
next_char = []


# Loop over range, stepping by 5
for i in range(0, len(encoded) - maxlen, step):
    # Append 40 char slices of `encoded` to sequences
    sequences.append(encoded[i : i + maxlen])
    # Append 
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1114068


In [33]:
type(encoded)

list

In [34]:
len(encoded)

5570380

In [35]:
len(data)

166768

In [36]:
encoded[1]

19

In [37]:
type(encoded[1])

int

In [38]:
# Make X & Y
# One-hot encode characters for each sequence in the string

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    x[i,t,char] = 1

  y[i, next_char[i]] = 1

## Model

In [39]:
# Build LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [41]:
def on_epoch_end(epoch, _):
  # Make function to print textgen after each epoch
  print()
  print('--- Generating text after Epoch: %d' % epoch)

  # Random start position to generate a sentence
  start_index = random.randint(0, len(text) - maxlen - 1)

  generated = ''

  # Use generated position to create start seed
  sentence = text[start_index: start_index + maxlen]
  generated += sentence

  print('---- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)

  for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_int[char]] = 1

      preds = model.predict(x_pred, verbose = 0)[0]
      next_index = sample(preds)
      next_char = int_char[next_index]

      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()
  
  print()

# Use defined function above to callback on epoch end
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [42]:
# Perform model fit

model.fit(x,
          y,
          batch_size=64,
          epochs=15,
          callbacks=[print_callback])

Epoch 1/15
17398/17408 [============================>.] - ETA: 0s - loss: 2.0728
--- Generating text after Epoch: 0
---- Generating with seed: " famish'd English, like pale ghosts,    "
 famish'd English, like pale ghosts,    ts.a’l  eeassl.rofitnale,:es saenaoweoT O..dtfiea,i.yo  aeeassys.,iey. ’yir   nfd' yas    ’y e ensao csirla;fs. !lnaioef.a  ai;;.o:a!.u ;idy! seaelo,.ra,i ,t. tey .!eo uslyw i t' sss yc io—o.Glsli? ehe.l.., unls s,a,,i  eusllst .-ec su o st  eess;ial,t et n;..r,  eo :.. e  toyeae. ;D.os..irsaaah;e!e sG. s'’.ks asqtsnlsa.a’etil: saa;o   '-lnl,.tlyyhs ; aysn ,—s] si_. ra ou ee!syr yi; ta d yaeesr e . n.’.Ias ee nynsslt,e.eee.me o’oa.s.otpge]esrsa.ga.;a.elaras  s sn,.nig’no a' w.]’esf’n ge.s ilsa: ,l sw ..]io saturloars..we. eleeay’y asng.as imf,.cieyqey,std . s  r sef:.l,sst  i-; . e,ue ef,tseel':au. ,.l, .see’aca,waolt.'ssn’’ ce    s,e t,. ri, :. .!;emeyioins.slae;?’ sytehs.osl;o .,sVo  'oy, il..nfs os ese ssrsc.e .e; le]Ba .u,ra;ene.elet.]ta. mcnstoi’ ..,eaee: a’t

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN